In [184]:
import numpy as np
import pandas as pd
import re

In [185]:
def analysis(nfs, app):
    print("""
====================
    CPU Analysis
====================""")
    
    df_cpu = pd.read_csv("{}-{}.cpu.csv".format(nfs, app), skiprows=14)
    df_dsk = pd.read_csv("{}-{}.dsk.csv".format(nfs, app), skiprows=14)
    df_numa = pd.read_csv("{}-{}.numa.csv".format(nfs, app), skiprows=14)
    df_tab = pd.read_csv("{}-{}.tab.csv".format(nfs, app), skiprows=14)

    df_cpu["Timestamp"] = pd.to_datetime(df_cpu["#Date"].map(str) + "-" + df_cpu["Time"], format="%Y%m%d-%H:%M:%S.%f")
    interval_time = df_cpu["Timestamp"][1] - df_cpu["Timestamp"][0]
    n_cpu = (len(df_cpu.columns) - 2) // 12
    cpus = [f"[CPU:{i}]" for i in range(n_cpu)]
    total_cpu = df_cpu[map(lambda x: x+"Totl%", cpus)]

    print(
        "Total CPU time [seconds]:\n",
        ((total_cpu >= 10 * 1).values.sum() * interval_time) / np.timedelta64(1, 's')
    )

    print(
        "Parallel CPU time [seconds]:\n", 
        (sum([1 for row in (total_cpu >= 10 * 1).values if any(row)]) * interval_time) / np.timedelta64(1, 's')
    )


    print(
        "Makes span [seconds]:\n",
        (df_cpu["Timestamp"][len(df_cpu["Timestamp"])-1] - df_cpu["Timestamp"][0]) / np.timedelta64(1, 's')
    )

    print("""
====================
    I/O Analysis
====================""")
    
    df_cpu = pd.read_csv("{}-{}.cpu.csv".format(nfs, app), skiprows=14)
    df_dsk = pd.read_csv("{}-{}.dsk.csv".format(nfs, app), skiprows=14)
    df_numa = pd.read_csv("{}-{}.numa.csv".format(nfs, app), skiprows=14)
    df_tab = pd.read_csv("{}-{}.tab.csv".format(nfs, app), skiprows=14)

    df_cpu["Timestamp"] = pd.to_datetime(df_cpu["#Date"].map(str) + "-" + df_cpu["Time"], format="%Y%m%d-%H:%M:%S.%f")
    interval_time = df_cpu["Timestamp"][1] - df_cpu["Timestamp"][0]

#     return df_tab
    if nfs == 'server':
        rw = df_dsk[['[DSK:vdc]RKBytes', '[DSK:vdc]WKBytes']]
        
    elif nfs == "client":
        rw = df_tab[['[NET]RxKBTot', '[NET]TxKBTot']]
       
    data_transfer = rw.sum(axis=0) / 1024
    print(
        'Data transefered [MB]:\n',
        data_transfer,
        data_transfer.sum()
    ) 
    print(
        'Total I/O time [seconds]:\n', 
        (rw.mask(rw > 0, 1).sum().sum()  * interval_time) / np.timedelta64(1, 's')
    )
    print(
        'Parallel I/O time [seconds]:\n',
        (sum([1 for row in (rw != 0).values if any(row)]) * interval_time) / np.timedelta64(1, 's')
    )
        

    

In [186]:
analysis('server', 'example')


    CPU Analysis
Total CPU time [seconds]:
 896.0
Parallel CPU time [seconds]:
 852.0
Makes span [seconds]:
 885.0

    I/O Analysis
Data transefered [MB]:
 [DSK:vdc]RKBytes    2527.796875
[DSK:vdc]WKBytes     462.996094
dtype: float64 2990.79296875
Total I/O time [seconds]:
 587.0
Parallel I/O time [seconds]:
 397.0


In [187]:
analysis('client', 'example')


    CPU Analysis
Total CPU time [seconds]:
 888.0
Parallel CPU time [seconds]:
 888.0
Makes span [seconds]:
 889.001

    I/O Analysis
Data transefered [MB]:
 [NET]RxKBTot    2521.138672
[NET]TxKBTot     468.042969
dtype: float64 2989.181640625
Total I/O time [seconds]:
 895.0
Parallel I/O time [seconds]:
 448.0
